In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import networkx as nx
import random
import jReversion as jR
import robustness as rb
import pandas as pd
import numpy as np
import par_helper as ph
import concurrent.futures
from LDOI import BooleanDOI_processing as BDOIp

In [2]:
def gen_random_boolean_network_er_kauffman(n=30, p=0.3, bias=0.5):
    '''

    The code is originated from the read_ndetwork() by Colin Campbell.
    '''
    def clean_states(x):
        #cleans binary representation of node input states
        out=x[2:]                                                               # Strip leading 0b
        return '0'*(len(inf)-len(out))+out                                      # Append leading 0's as needed

    while True:
        g = nx.generators.random_graphs.gnp_random_graph(n=n, p=p, directed=True)
        
        input_node = [node for node in g.nodes if g.in_degree(node) == 0]
        output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        if len(input_node) == 0:              # at least one input node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.in_edges(random_node)))
            input_node = [node for node in g.nodes if g.in_degree(node) == 0]
            
        if len(output_node) == 0:            # at least one output node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.out_edges(random_node)))
            output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        
        if len(g.subgraph(input_node + output_node).edges) > 0: continue # output nodes are connected to input nodes
                    
        if nx.is_weakly_connected(g): break

    for n in g.nodes:
        inf = list(g.predecessors(n))
        if len(inf) > 0: 
            g.nodes[n]['update_nodes'] = inf.copy()
            g.nodes[n]['update_rules'] = {}

            bool_states = map(bin,range(2**len(inf)))
            bool_states = map(clean_states,bool_states)
            for j in bool_states:
                g.nodes[n]['update_rules'][j] = int(random.random() < bias)    # Store outcome for every possible input
        else:
#             g.add_edge(n, n)
            g.nodes[n]['update_nodes'] = [n]
            g.nodes[n]['update_rules'] = {'0': 0, '1': 1}

    return g,list(g.nodes)

In [3]:
def r3_test_er_kauffman(n=30, p=0.3, bias=0.5, prefix='n', suffix='n', sampling_num=10):
    g, read_nodes = gen_random_boolean_network_er_kauffman(n=n, p=p, bias=bias)
    mapping = {}  # nodename to number index
    inverse_mapping = {}  # number index to nodename
    read_nodes_dict = {}
    inverse_read_nodes_dict = {}
    for i, node in enumerate(read_nodes):
        index = prefix + str(i) + suffix
        mapping[str(node)] = index
        inverse_mapping[index] = str(node)
        mapping['~' + str(node)] = '~' + index
        inverse_mapping['~' + index] = '~' + str(node)
        read_nodes_dict[i] = str(node)
        inverse_read_nodes_dict[str(node)] = i
    
    input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
    output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]
    
    num_inputs = len(input_nodes)
    num_input_conditions = 2 ** num_inputs
    input_conditions = np.ndarray((num_inputs, 2), dtype=object)
    for idx, input_node in enumerate(input_nodes):
        input_conditions[idx, 0] = '~' + str(input_node)
        input_conditions[idx, 1] = str(input_node)
    
    
    output_nodes_ex = []
    output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
    output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
    input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    
    GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])
    
    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
            else:
#                 nodeClass = 'C2'
                C2.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        else:
#             nodeClass = 'C3'
            C3.append(NODE)
#             canalizing = IIDC['iid_canalizing'][NODE]
#             unreachable = UN['input_unreachable'][NODE]
    

    robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
    robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)
    
    robustness = []
    for key, val in robustness_pa.items():
        robustness.append(val['oe'])
        robustness.append(val['ko'])
#     print(np.mean(robustness))
    
    
    # deterministic_IO
    empty_io = [x for x in IORelation[0,:] if len(x) == 0]
    len_io = [len(x) for x in IORelation[0,:]]
    empty_io_ratio = len(empty_io) / num_input_conditions
    deterministic_io_ratio = sum(len_io) / (num_input_conditions * len(output_nodes))
    
    
    LDOI_union = set()
    for LDOI in LDOIs[0,:]:
        LDOI_union = LDOI_union.union(LDOI)
    LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
    GSub = nx.subgraph(GExpanded, LDOI_union)
    io_count = 0
    io_pathway_count = 0
    for i in range(num_input_conditions):
        # G_copied = G_expanded.copy()
        BinaryBit = np.binary_repr(i, width=num_inputs)
        InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
        # source_nodes = input_condition + constant_nodes
        Sources = set([x for x in InputCondition])
        # for node in source:
        #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
        #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

        Targets = set([x for x in IORelation[0, i]])
        if len(Targets) > 0:
            for s in Sources:
                for t in Targets:
                    if nx.has_path(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
                        io_count += 1
                        len_shortest_path = nx.shortest_path_length(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
                        for p in nx.all_simple_paths(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
                            io_pathway_count += 1

    if io_count == 0:
        redundancy = -1
    else:
        redundancy = io_pathway_count/float(io_count)
    
    return {'size_of_network': len(g), 'num_link': len(g.edges), 'num_input': len(input_nodes), 'num_output': len(output_nodes),
            'C0': len(C0), 'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 
            'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip,
            'empty_io_ratio': empty_io_ratio, 'deterministic_io_ratio':deterministic_io_ratio,
            'redundancy': redundancy}

In [4]:
num_of_networks = 100
size_of_network = 20
link_probability = 0.1
sampling_num = 15
num_of_worker = 10
result_pd = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 
            'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
            'redundancy'])
with concurrent.futures.ProcessPoolExecutor(max_workers=num_of_worker) as executor:
    futures = {executor.submit(r3_test_er_kauffman, n=size_of_network, p=link_probability, sampling_num=10): network_idx for network_idx in range(num_of_networks)}

for future in concurrent.futures.as_completed(futures):
    network_idx = futures[future]
    temp = future.result()
    temp['network_idx'] = network_idx
    result_pd = result_pd.append(temp, ignore_index=True)

Process ForkProcess-2:
Process ForkProcess-8:
Process ForkProcess-1:
Process ForkProcess-10:
Process ForkProcess-9:
Process ForkProcess-3:
Process ForkProcess-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jijoo

KeyboardInterrupt: 

In [ ]:
result_pd.to_csv('../data/random_networks_er_kauffman_3.csv')

In [5]:
result_pd

,network_idx,size_of_network,num_link,C0,C1,C2,C3,robustness_pa,robustness_ip,redundancy


In [86]:
def gen_random_boolean_network_kout_kauffman(n=30, k=2, indeg_thr=6, alpha=1, bias=0.5):
    '''

    The code is originated from the read_ndetwork() by Colin Campbell.
    '''
    def clean_states(x):
        #cleans binary representation of node input states
        out=x[2:]                                                               # Strip leading 0b
        return '0'*(len(inf)-len(out))+out                                      # Append leading 0's as needed

    while True:
        g = nx.DiGraph(nx.generators.directed.random_k_out_graph(n=n, k=k, alpha=alpha, self_loops=False))
        
        input_node = [node for node in g.nodes if g.in_degree(node) == 0]
        output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        if len(input_node) == 0:              # at least one input node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.in_edges(random_node)))
            input_node = [node for node in g.nodes if g.in_degree(node) == 0]
            
        if len(output_node) == 0:            # at least one output node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.out_edges(random_node)))
            output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        
        if len(g.subgraph(input_node + output_node).edges) > 0: continue # output nodes are connected to input nodes
        if max(dict(g.in_degree).values()) > indeg_thr: continue            
        if nx.is_weakly_connected(g): break

    for n in g.nodes:
        inf = list(g.predecessors(n))
        if len(inf) > 0: 
            g.nodes[n]['update_nodes'] = inf.copy()
            g.nodes[n]['update_rules'] = {}

            bool_states = map(bin,range(2**len(inf)))
            bool_states = map(clean_states,bool_states)
            for j in bool_states:
                g.nodes[n]['update_rules'][j] = int(random.random() < bias)    # Store outcome for every possible input
        else:
#             g.add_edge(n, n)
            g.nodes[n]['update_nodes'] = [n]
            g.nodes[n]['update_rules'] = {'0': 0, '1': 1}

    return g,list(g.nodes)

In [72]:
def r3_test_kout_kauffman(n=30, k=2, alpha=1, bias=0.5, indeg_thr=6, prefix='n', suffix='n', sampling_num=10):
    g, read_nodes = gen_random_boolean_network_kout_kauffman(n=n, k=k, alpha=alpha, bias=bias, indeg_thr=indeg_thr)
    mapping = {}  # nodename to number index
    inverse_mapping = {}  # number index to nodename
    read_nodes_dict = {}
    inverse_read_nodes_dict = {}
    for i, node in enumerate(read_nodes):
        index = prefix + str(i) + suffix
        mapping[str(node)] = index
        inverse_mapping[index] = str(node)
        mapping['~' + str(node)] = '~' + index
        inverse_mapping['~' + index] = '~' + str(node)
        read_nodes_dict[i] = str(node)
        inverse_read_nodes_dict[str(node)] = i
    
    input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
    output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]
    
    num_inputs = len(input_nodes)
    num_input_conditions = 2 ** num_inputs
    input_conditions = np.ndarray((num_inputs, 2), dtype=object)
    for idx, input_node in enumerate(input_nodes):
        input_conditions[idx, 0] = '~' + str(input_node)
        input_conditions[idx, 1] = str(input_node)
    
    
    output_nodes_ex = []
    output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
    output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
    input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    
    GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])
    
    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
            else:
#                 nodeClass = 'C2'
                C2.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        else:
#             nodeClass = 'C3'
            C3.append(NODE)
#             canalizing = IIDC['iid_canalizing'][NODE]
#             unreachable = UN['input_unreachable'][NODE]
    

    robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
    robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)
    
    robustness = []
    for key, val in robustness_pa.items():
        robustness.append(val['oe'])
        robustness.append(val['ko'])
#     print(np.mean(robustness))
    
    # deterministic_IO
    empty_io = [x for x in IORelation[0,:] if len(x) == 0]
    len_io = [len(x) for x in IORelation[0,:]]
    empty_io_ratio = len(empty_io) / num_input_conditions
    deterministic_io_ratio = sum(len_io) / (num_input_conditions * len(output_nodes))
    
    
    LDOI_union = set()
    for LDOI in LDOIs[0,:]:
        LDOI_union = LDOI_union.union(LDOI)
    LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
    GSub = nx.subgraph(GExpanded, LDOI_union)
    io_count = 0
    io_pathway_count = 0
    for i in range(num_input_conditions):
        # G_copied = G_expanded.copy()
        BinaryBit = np.binary_repr(i, width=num_inputs)
        InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
        # source_nodes = input_condition + constant_nodes
        Sources = set([x for x in InputCondition])
        # for node in source:
        #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
        #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

        Targets = set([x for x in IORelation[0, i]])
        if len(Targets) > 0:
            for s in Sources:
                for t in Targets:
                    if nx.has_path(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
                        io_count += 1
                        len_shortest_path = nx.shortest_path_length(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
                        for p in nx.all_simple_paths(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
                            io_pathway_count += 1

    if io_count == 0:
        redundancy = -1
    else:
        redundancy = io_pathway_count/float(io_count)
    
    return {'size_of_network': len(g), 'num_link': len(g.edges), 'num_input': len(input_nodes), 'num_output': len(output_nodes),
            'C0': len(C0), 'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 
            'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip,
            'empty_io_ratio': empty_io_ratio, 'deterministic_io_ratio':deterministic_io_ratio,
            'redundancy': redundancy}

In [97]:
num_of_networks = 50
size_of_network = 15
k_out = 2
sampling_num = 10
num_of_worker = 5
result_pd = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 
            'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
            'redundancy'])
with concurrent.futures.ProcessPoolExecutor(max_workers=num_of_worker) as executor:
    futures = {executor.submit(r3_test_kout_kauffman, n=size_of_network, k=k_out, indeg_thr=6, sampling_num=sampling_num): network_idx for network_idx in range(num_of_networks)}

for future in concurrent.futures.as_completed(futures):
    network_idx = futures[future]
    temp = future.result()
    temp['network_idx'] = network_idx
    result_pd = result_pd.append(temp, ignore_index=True)

In [99]:
result_pd.to_csv('../data/random_networks_kout_kauffman_211014.csv')

In [98]:
result_pd

,network_idx,size_of_network,num_link,C0,C1,C2,C3,robustness_pa,robustness_ip,redundancy,deterministic_io_ratio,empty_io_ratio,num_input,num_output
0,4.0,15.0,22.0,15.0,0.0,0.0,0.0,0.733333,0.733333,-1.000000,0.00000,1.00000,4.0,1.0
1,23.0,15.0,27.0,15.0,0.0,0.0,0.0,0.313333,0.600000,-1.000000,0.00000,1.00000,4.0,1.0
2,34.0,15.0,26.0,12.0,1.0,0.0,2.0,0.566667,0.674674,2.666667,1.00000,0.00000,3.0,1.0
3,26.0,15.0,28.0,15.0,0.0,0.0,0.0,0.366667,0.541732,-1.000000,0.00000,1.00000,4.0,1.0
4,40.0,15.0,26.0,15.0,0.0,0.0,0.0,0.300000,0.600846,-1.000000,0.00000,1.00000,5.0,1.0
5,9.0,15.0,28.0,11.0,1.0,0.0,3.0,0.700000,0.645443,-1.000000,0.00000,1.00000,4.0,1.0
6,48.0,15.0,25.0,10.0,2.0,0.0,3.0,0.516667,0.724284,3.750000,1.00000,0.00000,4.0,1.0
7,30.0,15.0,27.0,15.0,0.0,0.0,0.0,0.666667,0.597526,-1.000000,0.00000,1.00000,4.0,1.0
8,21.0,15.0,24.0,15.0,0.0,0.0,0.0,0.575000,0.678776,-1.000000,0.00000,1.00000,4.0,1.0
9,37.0,15.0,27.0,0.0,4.0,0.0,11.0,0.366667,0.596745,6.500000,0.43750,0.56250,4.0,1.0
